# Exporting to BIDS Formats


The [Brain Imaging Data Structure (BIDS)](https://bids.neuroimaging.io/index.html) is a comprehensive framework designed to systematically organize and share diverse types of data, including behavioral, physiological, and neuroimaging information. Converting datasets into BIDS format is a widely adopted methodology, particularly in the process of curating datasets that adhere to the principles of FAIR (Findable, Accessible, Interoperable, Reusable).

For datasets encompassing mobile eye-tracking data, it is essential to apply specific BIDS specifications tailored for such data. In this context, Motion-BIDS and Eye-Tracking-BIDS specifications are noteworthy. Motion-BIDS ([BEP029](https://github.com/bids-standard/bids-specification/pull/981)) has been successfully integrated into the official BIDS specification, demonstrating its readiness for use in organizing motion-related data. On the other hand, Eye-Tracking-BIDS ([BEP020](https://github.com/bids-standard/bids-specification/pull/1128)) is still undergoing development, reflecting ongoing efforts to provide a standardized format for eye-tracking data. You can find more information about these specifications in the following references:

> <cite>Gorgolewski, K., Auer, T., Calhoun, V. et al. The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments. Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44<cite>

> <cite>Jeung, S., Cockx, H., Appelhoff, S. et al. Motion-BIDS: an extension to the brain imaging data structure to organize motion data for reproducible research. Sci Data 11, 716 (2024). https://doi.org/10.1038/s41597-024-03559-8<cite>

In the ensuing section, we will delve into the procedure for exporting data to Motion-BIDS. This will be accomplished using the `export_motion_bids` method available within PyNeon's `Recording` objects, offering a practical guide for researchers aiming to standardize their motion data in alignment with the BIDS framework.

In [1]:
import json
from pathlib import Path
import pandas as pd
from pyneon import Dataset, get_sample_data

dataset = Dataset(get_sample_data("markers", format="cloud"))
rec = dataset.recordings[0]

## Exporting to Motion-BIDS

To use `export_motion_bids` method, we need to specify the output directory where the BIDS dataset will be saved, and a string prefix to denote this session of data. The prefix follows the following format (Fields in [] are optional):

```text
sub-<label>[_ses-<label>]_task-<label>_tracksys-<label>[_acq-<label>][_run-<index>]
```

If you have any additional metadata that you would like to include, you can pass it as a dictionary to the `extra_metadata` argument. This metadata will be saved in the `dataset_description.json` file.

Let's see what files will be exported to the BIDS dataset directory:

In [2]:
# Create a BIDS directory
motion_dir = Path("export") / "BIDS" / "sub-1" / "motion"

# Export the motion data to BIDS format
prefix = "sub-1_task-LabMuse_tracksys-NeonIMU_run-1"
extra_metadata = {
    "TaskName": "LabMuse",
    "InstitutionName": "Streeling University",
    "InstitutionAddress": "Trantor, Galactic Empire",
    "InstitutionalDepartmentName": "Department of Psychohistory",
}

rec.export_motion_bids(motion_dir, prefix=prefix, extra_metadata=extra_metadata)

# Print all the conents of motion_dir
for path in motion_dir.iterdir():
    print(path.name)

sub-1_task-LabMuse_tracksys-NeonIMU_run-1_channels.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_channels.tsv
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_motion.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_motion.tsv
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physio.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physio.tsv.gz
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physioevents.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physioevents.tsv


The contents of these files follow the Motion-BIDS specification at: https://bids-specification.readthedocs.io/en/stable/modality-specific-files/motion.html.

For example, the `_motion.tsv` is a tab-separated values file that contains the (n_samples, n_channels) motion data without a header:

In [3]:
physio_tsv_path = motion_dir / f"{prefix}_motion.tsv"
physio_df = pd.read_csv(physio_tsv_path, sep="\t")
print(f"Motion data shape: {physio_df.shape}")
print(physio_df.head())

Motion data shape: (25104, 13)
   -51.717758  -2.668381  60.407639  -0.239258  -0.118164  0.899902  \
0  -58.860779  -2.059937  67.914963  -0.230957  -0.143066  0.915527   
1  -65.940857  -1.632690  75.727463  -0.208008  -0.144043  0.927246   
2  -76.131821   0.989914  89.948654  -0.208008  -0.158691  0.926758   
3  -79.427719   3.431320  95.075607  -0.187012  -0.171875  0.937012   
4  -81.747055   5.998611  99.592209  -0.150879  -0.189453  0.945801   

   7.143753693332938  -3.1793275574051094  -106.20521917582651  \
0           7.216643            -3.664375          -105.502129   
1           7.297498            -4.196307          -104.722450   
2           7.394377            -4.766157          -103.867197   
3           7.515937            -5.370411          -102.939744   
4           7.657661            -5.993968          -101.953045   

   0.5976047515869141  0.0331803187727928  0.0595318600535392  \
0            0.602169            0.030758            0.063474   
1            0.

Its metadata is stored in the `_motion.json` file, which contains (note the extra metadata we added):

In [4]:
motion_json = motion_dir / f"{prefix}_motion.json"
with open(motion_json, "r") as f:
    motion_metadata = json.load(f)
print(json.dumps(motion_metadata, indent=4))

{
    "TaskName": "LabMuse",
    "TaskDescription": "",
    "Instructions": "",
    "DeviceSerialNumber": "114837",
    "Manufacturer": "TDK InvenSense & Pupil Labs",
    "ManufacturersModelName": "ICM-20948",
    "SoftwareVersions": "App version: 2.9.26-prod; Pipeline version: 2.8.0",
    "InstitutionName": "Streeling University",
    "InstitutionAddress": "Trantor, Galactic Empire",
    "InstitutionalDepartmentName": "Department of Psychohistory",
    "SamplingFrequency": 110,
    "ACCELChannelCount": 3,
    "GYROChannelCount": 3,
    "MissingValues": "n/a",
    "MotionChannelCount": 13,
    "ORNTChannelCount": 7,
    "SubjectArtefactDescription": "",
    "TrackedPointsCount": 0,
    "TrackingSystemName": "IMU included in Neon"
}


The metadata for each channel (each column in the `_motion.tsv` file) is stored in `_channels.tsv` file:

In [5]:
channels_tsv_path = motion_dir / f"{prefix}_channels.tsv"
channels_df = pd.read_csv(channels_tsv_path, sep="\t")
print(channels_df)

              name component   type tracked_point      units  \
0           gyro x         x   GYRO          Head      deg/s   
1           gyro y         y   GYRO          Head      deg/s   
2           gyro z         z   GYRO          Head      deg/s   
3   acceleration x         x  ACCEL          Head          g   
4   acceleration y         y  ACCEL          Head          g   
5   acceleration z         z  ACCEL          Head          g   
6             roll         x   ORNT          Head        deg   
7            pitch         y   ORNT          Head        deg   
8              yaw         z   ORNT          Head        deg   
9     quaternion w         w   ORNT          Head  arbitrary   
10    quaternion x         x   ORNT          Head  arbitrary   
11    quaternion y         y   ORNT          Head  arbitrary   
12    quaternion z         z   ORNT          Head  arbitrary   

    sampling_frequency  
0                  103  
1                  103  
2                  103  
3  

Finally, the `_channels.json` file contains the coordinate system:

In [6]:
channels_json_path = motion_dir / f"{prefix}_channels.json"
with open(channels_json_path, "r") as f:
    channels_metadata = json.load(f)
print(json.dumps(channels_metadata, indent=4))

{
    "reference_frame": {
        "Levels": {
            "global": {
                "SpatialAxes": "RAS",
                "RotationOrder": "ZXY",
                "RotationRule": "right-hand",
                "Description": "This global reference frame is defined by the IMU axes: X right, Y anterior, Z superior. The scene camera frame differs from this frame by a 102-degree rotation around the X-axis. All motion data are expressed relative to the IMU frame for consistency."
            }
        }
    }
}


## Exporting to Eye-Tracking-BIDS



In [7]:
rec.export_eye_bids(motion_dir, prefix=prefix)
# Print all the conents of motion_dir
for path in motion_dir.iterdir():
    print(path.name)

C:\Users\qian.chu\Documents\GitHub\PyNeon\pyneon\preprocess\preprocess.py:67: UserWarning: 23 out of 48219 requested timestamps are outside the data time range and will have empty data.
  warn(


sub-1_task-LabMuse_tracksys-NeonIMU_run-1_channels.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_channels.tsv
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_motion.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_motion.tsv
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physio.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physio.tsv.gz
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physioevents.json
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physioevents.tsv
sub-1_task-LabMuse_tracksys-NeonIMU_run-1_physioevents.tsv.gz


C:\Users\qian.chu\Documents\GitHub\PyNeon\pyneon\export\export_bids.py:233: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  physioevents_data = pd.concat([physioevents_data, events_data], ignore_index=True)


In [8]:
physio_tsv_path = motion_dir / f"{prefix}_physio.tsv.gz"
physio_df = pd.read_csv(physio_tsv_path, sep="\t", compression="gzip")
print(f"Eye-tracking data shape: {physio_df.shape}")
print(physio_df.head())

Eye-tracking data shape: (48218, 5)
   1758493516475391023   535.36  1016.989  4.6517  4.5655
0  1758493516480394023  519.555  1033.236  4.0452  4.5209
1  1758493516485390023  521.541  1045.861  3.4693  4.5801
2  1758493516490390023  503.323  1049.000  4.1283  4.7350
3  1758493516495406023  498.538  1065.085  4.5646  4.8635
4  1758493516500390023  485.796  1069.292  4.7579  4.8452


In [9]:
physio_json = motion_dir / f"{prefix}_physio.json"
with open(physio_json, "r") as f:
    physio_metadata = json.load(f)
print(json.dumps(physio_metadata, indent=4))

{
    "SamplingFrequency": 199.71919360432685,
    "StartTime": 0,
    "Columns": [
        "timestamp",
        "x_coordinate",
        "y_coordinate",
        "left_pupil_diameter",
        "right_pupil_diameter"
    ],
    "DeviceSerialNumber": "114837",
    "Manufacturer": "Pupil Labs",
    "ManufacturersModelName": "Neon",
    "SoftwareVersions": "App version: 2.9.26-prod; Pipeline version: 2.8.0",
    "PhysioType": "eyetrack",
    "EnvironmentCoorinates": "top-left",
    "RecordedEye": "cyclopean",
    "SampleCoordinateSystem": "gaze-in-world",
    "EyeTrackingMethod": "real-time neural network",
    "timestamp": {
        "Description": "UTC timestamp in nanoseconds of the sample",
        "Units": "ns"
    },
    "x_coordinate": {
        "LongName": "Gaze position (x)",
        "Description": "Horizontal gaze position x-coordinate in the scene camera frame, measured from the top-left corner",
        "Units": "pixel"
    },
    "y_coordinate": {
        "LongName": "Gaze posit

In [10]:
physioevents_tsv_path = motion_dir / f"{prefix}_physioevents.tsv.gz"
physioevents_df = pd.read_csv(physioevents_tsv_path, sep="\t", compression="gzip")
print(f"Eye-tracking data shape: {physioevents_df.shape}")
print(physioevents_df.head())

Eye-tracking data shape: (1101, 4)
                 onset  duration trial_type          message
0  1758493514096000000       NaN        NaN  recording.begin
1  1758493516475391023     0.105    saccade              NaN
2  1758493516580513023     0.535   fixation              NaN
3  1758493517116002023     0.110    saccade              NaN
4  1758493517226130023     0.135   fixation              NaN


In [11]:
physioevents_json = motion_dir / f"{prefix}_physioevents.json"
with open(physioevents_json, "r") as f:
    physioevents_metadata = json.load(f)
print(json.dumps(physioevents_metadata, indent=4))

{
    "Columns": [
        "onset",
        "duration",
        "trial_type",
        "message"
    ],
    "Description": "Eye events and messages logged by Neon",
    "OnsetSource": "timestamp",
    "onset": {
        "Description": "UTC timestamp in nanoseconds of the start of the event",
        "Units": "ns"
    },
    "duration": {
        "Description": "Event duration in seconds",
        "Units": "s"
    },
    "trial_type": {
        "Description": "Type of trial event",
        "Levels": {
            "fixation": {
                "Description": "Fixation event"
            },
            "saccade": {
                "Description": "Saccade event"
            },
            "blink": {
                "Description": "Blink event"
            }
        }
    }
}
